In [1]:
import sys
sys.path.insert(0, '../')

In [2]:
import pandas as pd
import numpy as np
from text_eval.similarity_eval import calculate_similarity

c:\Users\U9173668\python\translation_eval\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.43819512537676897


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\U9173668\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Load the DataFrame
df = pd.read_csv('data/input/data_example.csv')
df.head()

,original_French,translation_3_French,translation_5_French,original_German,translation_3_German,translation_5_German
0,Bonjour le monde. Aujourd'hui est un beau jour...,Salut le monde. C'est un magnifique jour pour ...,Bonjour le monde. Aujourd'hui est un beau jour...,"Hallo Welt. Heute ist ein schöner Tag, um etwa...","Hallo, Welt. Es ist ein wunderbarer Tag, um Ne...","Grüße, Welt. Dieser Tag eignet sich hervorrage..."
1,Je t'aime. Tu es la lumière de ma vie.,Je vous aime beaucoup. Vous êtes le soleil de ...,Je t'aime. Tu es la lumière de ma vie.,Ich liebe dich. Du bist das Licht meines Lebens.,Ich hab dich lieb. Du bist die Sonne in meinem...,Ich verehre dich. Du erleuchtest mein Leben wi...
2,Merci beaucoup pour votre aide. Cela signifie ...,Un grand merci pour votre soutien. Cela compte...,Merci infiniment pour tout. Votre aide est ine...,Vielen Dank für Ihre Hilfe. Das bedeutet mir s...,Herzlichen Dank für Ihre Unterstützung. Das is...,Tausend Dank für alles. Ihre Hilfe ist mir uns...


In [6]:
# Initialize containers for the results
detailed_results = {}
summary_results = {}

# Choose the method: 'bert' or 'bleu'
method = 'bleu'  # Or 'bert', depending on what you want to use

# Process DataFrame to identify languages and calculate similarity
languages = set(col.split('_')[-1] for col in df.columns if col.startswith('original'))

for lang in languages:
    original_col = f'original_{lang}'
    translation_cols = [col for col in df.columns if col.endswith(lang) and col.startswith('translation')]
    
    for t_col in translation_cols:
        model_number = t_col.split('_')[1]
        detailed_key = f'Model {model_number} - {lang}'
        summary_key = f'Model {model_number} - {lang} - Average'
        
        # Calculate similarities for each row
        similarities = []
        for index, row in df.iterrows():
            if pd.notnull(row[t_col]) and pd.notnull(row[original_col]):
                similarity = calculate_similarity(row[t_col], row[original_col], method=method)
                similarities.append(similarity)
                # Store detailed result
                detailed_results.setdefault(detailed_key, []).append(similarity)
                
        # Calculate and store the average similarity
        average_similarity = np.mean(similarities)
        summary_results[summary_key] = average_similarity

c:\Users\U9173668\python\translation_eval\venv\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\U9173668\python\translation_eval\venv\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\U9173668\python\translation_eval\venv\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Con

In [7]:
# Convert results to DataFrames for easy saving and viewing
detailed_results_df = pd.DataFrame.from_dict(detailed_results, orient='index').transpose()
summary_results_df = pd.DataFrame(list(summary_results.items()), columns=['Model-Language', 'Average Similarity'])

# Save the detailed and summary results to Excel files
detailed_results_df.to_excel('data/output/detailed_similarity_results.xlsx', index=False)
summary_results_df.to_excel('data/output/summary_similarity_results.xlsx', index=False)

# Optionally, display the results
print("Detailed Results:")
print(detailed_results_df.head())
print("\nSummary Results:")
print(summary_results_df)

Detailed Results:
   Model 3 - French  Model 5 - French  Model 3 - German  Model 5 - German
0      3.550619e-78      6.100034e-01      4.601359e-78     7.090820e-155
1     1.179540e-231      1.000000e+00      3.378781e-78      3.535461e-78
2     6.624397e-155      3.919446e-78      3.532876e-78     8.512571e-155

Summary Results:
               Model-Language  Average Similarity
0  Model 3 - French - Average        1.183540e-78
1  Model 5 - French - Average        5.366678e-01
2  Model 3 - German - Average        3.837672e-78
3  Model 5 - German - Average        1.178487e-78
